이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=25a1a24d191649865a04f61ecc1ba6cf8dbaf427983bf22e9f8d420dc83a5618
  Stored in directory: /root/.cache/pip/wheels/07/a0/a3/d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1


In [2]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/orders.csv

--2025-11-21 08:01:40--  https://s3-geospatial.s3.us-west-2.amazonaws.com/orders.csv
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.92.154.146, 52.92.138.122, 3.5.76.107, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.92.154.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89951 (88K) [text/csv]
Saving to: ‘orders.csv’

orders.csv          100%[===================>]  87.84K  --.-KB/s    in 0.07s   

2025-11-21 08:01:40 (1.22 MB/s) - ‘orders.csv’ saved [89951/89951]



**Spark Session**

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark Hive") \
    .enableHiveSupport() \
    .getOrCreate()

In [4]:
!ls -tl

total 92
drwxr-xr-x 1 root root  4096 Nov 17 14:29 sample_data
-rw-r--r-- 1 root root 89951 Apr 24  2022 orders.csv


In [5]:
# Read in data and create a DataFrame
df = spark.read.csv("orders.csv", inferSchema=True, header=True, sep ='\t')

In [6]:
df.show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [7]:
spark.sql("CREATE DATABASE IF NOT EXISTS TEST_DB")
spark.sql("USE TEST_DB")

DataFrame[]

In [8]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|  test_db|
+---------+



In [9]:
!ls -tl

total 104
drwxr-xr-x 3 root root  4096 Nov 21 08:04 spark-warehouse
drwxr-xr-x 5 root root  4096 Nov 21 08:04 metastore_db
-rw-r--r-- 1 root root   667 Nov 21 08:04 derby.log
drwxr-xr-x 1 root root  4096 Nov 17 14:29 sample_data
-rw-r--r-- 1 root root 89951 Apr 24  2022 orders.csv


In [10]:
df.write.saveAsTable("TEST_DB.orders", mode="overwrite")

In [11]:
!ls -tl spark-warehouse/test_db.db/orders/

total 24
-rw-r--r-- 1 root root     0 Nov 21 08:07 _SUCCESS
-rw-r--r-- 1 root root 23051 Nov 21 08:07 part-00000-ea3d5455-deae-4216-8457-471895f585f8-c000.snappy.parquet


In [12]:
spark.sql("SELECT * FROM TEST_DB.orders").show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [13]:
spark.table("TEST_DB.orders").show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [14]:
!ls -tl metastore_db/

total 28
drwxr-xr-x 2 root root 4096 Nov 21 08:07 seg0
-rw-r--r-- 1 root root  900 Nov 21 08:04 service.properties
-rw-r--r-- 1 root root  608 Nov 21 08:04 README_DO_NOT_TOUCH_FILES.txt
drwxr-xr-x 2 root root 4096 Nov 21 08:04 log
-rw-r--r-- 1 root root   38 Nov 21 08:04 db.lck
-rw-r--r-- 1 root root    4 Nov 21 08:04 dbex.lck
drwxr-xr-x 2 root root 4096 Nov 21 08:04 tmp


In [15]:
spark.catalog.listTables()

[Table(name='orders', catalog='spark_catalog', namespace=['test_db'], description=None, tableType='MANAGED', isTemporary=False)]

In [16]:
spark.sql("""
    DROP TABLE IF EXISTS TEST_DB.orders_count;
""")

DataFrame[]

In [17]:
spark.sql("""
    CREATE TABLE TEST_DB.orders_count AS
    SELECT order_id, COUNT(1) as count
    FROM TEST_DB.orders
    GROUP BY 1""")

DataFrame[]

In [18]:
spark.catalog.listTables()

[Table(name='orders', catalog='spark_catalog', namespace=['test_db'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='orders_count', catalog='spark_catalog', namespace=['test_db'], description=None, tableType='MANAGED', isTemporary=False)]

In [19]:
!ls -tl spark-warehouse/test_db.db/

total 8
drwxr-xr-x 2 root root 4096 Nov 21 08:09 orders_count
drwxr-xr-x 2 root root 4096 Nov 21 08:07 orders


In [20]:
spark.sql("SELECT * FROM test_db.orders_count LIMIT 10").show()

+-------------+-----+
|     order_id|count|
+-------------+-----+
|1209310019796|    1|
|1253925257428|    1|
|1314797846740|    1|
|1370637402324|    1|
|1967690285268|    1|
|1971226443988|    1|
|2106987970772|    1|
|2135281533140|    1|
|1271771070676|    1|
|1713331765460|    1|
+-------------+-----+

